# SynthMags_LSSTComCam_c26202

Based on Lynne Jones code here: https://rubin-obs.slack.com/archives/C0824CTA335/p1732311332938929

See also:  https://github.com/lsst/rubin_sim_notebooks/blob/main/photometry/calculating_magnitudes.ipynb

Created: 2024.12.05

## 1. Initial Setup...

### 1.1 Import useful python packages

In [ ]:
import pylab as plt
import warnings
import os
import numpy
import pandas as pd
from astropy.io import fits
from astropy import units as u
import rubin_sim.phot_utils as pt
import syseng_throughputs as st

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)

# Set filter warnings to "ignore" to avoid a lot of "logorrhea" to the screen:
warnings.filterwarnings("ignore")

### 1.2 Include user input

In [ ]:
# Set environment variable to point to location of the rubin_sim_data 
#  (per Lynne Jones' Slack message on the #sciunit-photo-calib channel from 26 Nov 2024):
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# Which CalSpec C26202 spectrum FITS file to to use?
sedfile = '~/Downloads/c26202_stiswfcnic_007.fits'
#sedfile = '~/Downloads/c26202_mod_007.fits'
#sedfile = '~/Downloads/c26202_stiswfcnic_006.fits'
#sedfile = '~/Downloads/c26202_mod_008.fits'
#sedfile = '~/Downloads/c26202_stiswfcnic_007.fits'


# Plot symbol colors to use for ugrizy
plot_filter_colors_white_background = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00', 'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

### 1.3 Define useful classes and functions

In [ ]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

# 2 Calculate synthetic magnitudes

### 2.1 Read the file 


In [ ]:
seddata = fits.getdata(sedfile)

### 2.2 Translate to rubin_sim format

In [ ]:
wavelen = seddata['WAVELENGTH'] * u.angstrom.to(u.nanometer) # This is in angstroms - need in nanometers
flambda = seddata['FLUX'] / (u.angstrom.to(u.nanometer)) # this is in erg/sec/cm^^2/ang but we want /nm 

sed = pt.Sed(wavelen=wavelen, flambda=flambda)

### 2.3 Change detectors from (default) LSST to ComCam

In [ ]:
defaultDirs = st.setDefaultDirs()
defaultDirs['detector'] = defaultDirs['detector'].replace('/joint_minimum', '/itl')
hardware, system = st.buildHardwareAndSystem(defaultDirs)


### 2.4 Define the photometric paramters to use.

In [ ]:
phot_params = pt.PhotometricParameters(exptime=30, nexp=1, gain=1.0)

### 2.5 Calculate synthetic mags (and counts in electrons)

In [ ]:
mags = {}
counts = {}
counts_100k = {}
for f in 'ugrizy':
    mags[f] = sed.calc_mag(system[f])
    counts[f] = sed.calc_adu(system[f], phot_params)
    counts_100k[f] = counts[f]/100000

#pd.DataFrame([mags, counts, counts_100k], index=['AB mag', 'total counts', 'counts(100k)'])
for f in 'ugrizy':
    print(f, mags[f], counts[f], counts_100k[f])

In [ ]:
# Stop here
raise StopExecution